# PyABSA

### ABSA Instruction - Extract aspect term, category, opinion & sentiment polarity

### Atempting to infer on sample sentences

In [2]:
from pyabsa import ABSAInstruction

def absa(text):
  generator = ABSAInstruction.ABSAGenerator("multilingual")
  example = [
      "The food is good, but the service is bad.",
  ]
  
  if text:
    return generator.predict(text)
  
  return generator.predict(example)

In [3]:
absa("Avoid this place!")

[2024-03-20 20:13:58] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-03-20 20:13:58] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-03-20 20:13:58] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-03-20 20:13:58] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-03-20 20:13:58] (2.4.1.post1) Checkpoint already downloaded, skip
{'text': 'Avoid this place!', 'Quadruples': [{'aspect': 'place', 'polarity': 'negative', 'opinion': 'Avoid', 'category': 'RESTAURANT#GENERAL'}]}


{'text': 'Avoid this place!',
 'Quadruples': [{'aspect': 'place',
   'polarity': 'negative',
   'opinion': 'Avoid',
   'category': 'RESTAURANT#GENERAL'}]}

### Attempting to train and finetune model

Referencing code in https://github.com/yangheng95/PyABSA/blob/v2/pyabsa/tasks/ABSAInstruction/multitask_train.py


In [6]:
# -*- coding: utf-8 -*-
# file: train.py
# time: 11:30 2023/3/13
# author: YANG, HENG <hy345@exeter.ac.uk> (杨恒)
# github: https://github.com/yangheng95
# huggingface: https://huggingface.co/yangheng
# google scholar: https://scholar.google.com/citations?user=NPq5a_0AAAAJ&hl=en
# Copyright (C) 2019-2023. All Rights Reserved.
import os
import warnings

import findfile
from pyabsa import ABSAInstruction as absa_instruction

warnings.filterwarnings("ignore")
import pandas as pd


task_name = "multitask"
experiment_name = "ABSA quadruple extraction instruction"
# model_checkpoint = 'allenai/tk-instruct-base-def-pos'
model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"
# model_checkpoint = 'allenai/tk-instruct-large-def-pos'
# model_checkpoint = 'allenai/tk-instruct-3b-def-pos'
# model_checkpoint = 'google/mt5-base'

print("Experiment Name: ", experiment_name)
model_out_path = "checkpoints"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

# Load the data
# id_train_file_path = './integrated_datasets'
# id_test_file_path = './integrated_datasets'
# id_train_file_path = "./integrated_datasets/acos_datasets/"
# id_test_file_path = "./integrated_datasets/acos_datasets"
# id_train_file_path = "./integrated_datasets/acos_datasets/501.Laptop14"
# id_test_file_path = "./integrated_datasets/acos_datasets/501.Laptop14"
id_train_file_path = './integrated_datasets/acos_datasets/504.Restaurant16'
id_test_file_path = './integrated_datasets/acos_datasets/504.Restaurant16'


id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "train")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

id_tr_df = pd.DataFrame(id_tr_df)
id_te_df = pd.DataFrame(id_te_df)

loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

# Training arguments
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 2,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    # 'fp16': True,
    "fp16": False,
}

# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)


# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="train",
    batch_size=16,
)
id_tr_labels = [i.strip() for i in id_ds["train"]["labels"]]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="test",
    batch_size=16,
)
id_te_labels = [i.strip() for i in id_ds["test"]["labels"]]

# # Compute Metrics
# metrics = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
# print('----------------------- Training Set Metrics -----------------------')
# print(metrics)
#
# metrics = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
# print('----------------------- Testing Set Metrics -----------------------')
# print(metrics)
# Compute Metrics
metrics = t5_exp.get_classic_metrics(id_tr_labels, id_tr_pred_labels)
print("----------------------- Classic Training Set Metrics -----------------------")
print(metrics)

metrics = t5_exp.get_classic_metrics(id_te_labels, id_te_pred_labels)
print("----------------------- Classic Testing Set Metrics -----------------------")
print(metrics)

Experiment Name:  ABSA quadruple extraction instruction
Model output path:  checkpoints\multitask\allenaitk-instruct-large-def-pos-ABSA quadruple extraction instruction
[2024-03-20 16:50:39] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-03-20 16:50:39] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-03-20 16:50:39] (2.4.1.post1) Checkpoint:allenai/tk-instruct-large-def-pos is not found, you can raise an issue for requesting shares of checkpoints
[2024-03-20 16:50:39] (2.4.1.post1) No checkpoint found in Model Hub for task: allenai/tk-instruct-large-def-pos


KeyboardInterrupt: 

### Triple Extraction Task

#### Extracts Aspect term and sentiment polarity

In [4]:
from pyabsa import AspectSentimentTripletExtraction as ASTE

def aste_predict(text):
  # Load the model
  triplet_extractor = ASTE.AspectSentimentTripletExtractor("english")
  # triplet_extractor = ASTE.AspectSentimentTripletExtractor("multilingual")

  if text:
    return triplet_extractor.predict(text)
  # # Predict
  examples = [
      "I would like to have volume buttons rather than the adjustment that is on the front .####[([5, 6], [2], 'NEG')]",
      "It runs perfectly .####[([1], [2], 'POS')]",
      "Sometimes the screen even goes black on this computer .####[([2], [5], 'NEG')]",
      "Its fast and another thing I like is that it has three USB ports .####[([12, 13], [6], 'POS')]",
  ]
  for example in examples:
      triplet_extractor.predict(example)

In [ ]:
aste_predict("Avoid this place!")

#### Using CUDA

In [2]:
# -*- coding: utf-8 -*-
# file: trainer.py
# time: 2021/5/26 0026
# author: YANG, HENG <hy345@exeter.ac.uk> (杨恒)
# github: https://github.com/yangheng95
# Copyright (C) 2021. All Rights Reserved.

########################################################################################################################
#                    train and evaluate on your own apc_datasets (need train and test apc_datasets)                    #
########################################################################################################################

from pyabsa import (
    ModelSaveOption,
    DeviceTypeOption,
    DatasetItem,
)

from pyabsa import AspectSentimentTripletExtraction as ASTE


if __name__ == "__main__":
    config = ASTE.ASTEConfigManager.get_aste_config_english()
    config.max_seq_len = 120
    config.log_step = -1
    config.pretrained_bert = "microsoft/mdeberta-v3-base"
    # config.pretrained_bert = "microsoft/deberta-v3-base"
    # config.pretrained_bert = "bert-base-chinese"
    config.num_epoch = 16
    config.learning_rate = 2e-5
    # config.cache_dataset = False
    config.use_amp = True
    config.cache_dataset = True
    config.spacy_model = "zh_core_web_sm"

    # dataset = "Laptop14"
    # dataset = "aste"
    dataset = "semeval"
    # dataset = "chinese"
    trainer = ASTE.ASTETrainer(
        config=config,
        dataset=dataset,
        # from_checkpoint='english',
        checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
        # checkpoint_save_mode=ModelSaveOption.DO_NOT_SAVE_MODEL,
        auto_device=True,
    )
    triplet_extractor = trainer.load_trained_model()

    triplet_extractor.predict("The chicken rice is good but the service is lacking.")

[2024-03-21 01:41:49] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






C:\Users\regan\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


[2024-03-21 01:42:00] (2.4.1.post1) Set Model Device: cuda:0
[2024-03-21 01:42:00] (2.4.1.post1) Device Name: NVIDIA GeForce RTX 3060 Ti


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

2024-03-21 01:42:00,676 INFO: PyABSA version: 2.4.1.post1
2024-03-21 01:42:00,676 INFO: Transformers version: 4.38.2
2024-03-21 01:42:00,676 INFO: Torch version: 2.2.1+cu121+cuda12.1
2024-03-21 01:42:00,676 INFO: Device: NVIDIA GeForce RTX 3060 Ti
2024-03-21 01:42:00,699 INFO: Searching dataset semeval in local disk


...

[2024-03-21 03:00:41] (2.4.1.post1) Loading best model: checkpoints/emcgcn_custom_dataset_f1_69.74/ and evaluating on test set 
2024-03-21 03:00:51,461 INFO: 
--------------------------------------------------------------- Raw Metric Records ---------------------------------------------------------------
╒═════════════╤══════════════════════════════════════════════════╤═════════════════════╤═══════════╤══════════╤═══════╤═══════╤═════════╤═════════╕
│ Metric      │ Trial                                            │ Values              │  Average  │  Median  │  Std  │  IQR  │   Min   │   Max   │
╞═════════════╪══════════════════════════════════════════════════╪═════════════════════╪═══════════╪══════════╪═══════╪═══════╪═════════╪═════════╡
│ Max-Test-F1 │ emcgcn-custom_dataset-microsoft/mdeberta-v3-base │ [69.74240676662822] │  69.7424  │ 69.7424  │   0   │   0   │ 69.7424 │ 69.7424 │
╘═════════════╧══════════════════════════════════════════════════╧═════════════════════╧═══════════╧══

c:\Users\regan\Documents\FYP-FIT3162\myenv\Lib\site-packages\metric_visualizer\utils.py:31: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  self.skewness = stats.skew(self.data, keepdims=True)
c:\Users\regan\Documents\FYP-FIT3162\myenv\Lib\site-packages\pyabsa\framework\trainer_class\trainer_template.py:250: ResourceWarning: unclosed file <_io.TextIOWrapper name='c:\\Users\\regan\\Documents\\FYP-FIT3162\\Pyabsa\\absa_model\\logs\\emcgcn_20240321 014144\\trainer.log' mode='a' encoding='utf8'>
  self.config.logger.removeHandler(self.config.logger.handlers[0])


[2024-03-21 03:00:56] (2.4.1.post1) Load sentiment classifier from checkpoints/emcgcn_custom_dataset_f1_69.74/
[2024-03-21 03:00:56] (2.4.1.post1) config: checkpoints/emcgcn_custom_dataset_f1_69.74/emcgcn.config
[2024-03-21 03:00:56] (2.4.1.post1) state_dict: checkpoints/emcgcn_custom_dataset_f1_69.74/emcgcn.state_dict
[2024-03-21 03:00:56] (2.4.1.post1) model: None
[2024-03-21 03:00:56] (2.4.1.post1) tokenizer: checkpoints/emcgcn_custom_dataset_f1_69.74/emcgcn.tokenizer
[2024-03-21 03:00:56] (2.4.1.post1) Set Model Device: cuda:0
[2024-03-21 03:00:56] (2.4.1.post1) Device Name: NVIDIA GeForce RTX 3060 Ti


c:\Users\regan\Documents\FYP-FIT3162\myenv\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[2024-03-21 03:01:03] (2.4.1.post1) Batch: 0 {'sentence_id': 0, 'sentence': 'The chicken rice is good but the service is lacking.', 'Triplets': [{'Aspect': 'chicken rice', 'Opinion': 'good', 'Polarity': 'Positive'}, {'Aspect': 'chicken rice', 'Opinion': 'lacking.', 'Polarity': 'Negative'}, {'Aspect': 'service', 'Opinion': 'good', 'Polarity': 'Positive'}, {'Aspect': 'service', 'Opinion': 'lacking.', 'Polarity': 'Negative'}], 'True Triplets': []}
[2024-03-21 03:01:03] (2.4.1.post1) Batch: 0 {'sentence_id': 0, 'sentence': 'The chicken rice is good but the service is lacking.', 'Triplets': [{'Aspect': 'chicken rice', 'Opinion': 'good', 'Polarity': 'Positive'}, {'Aspect': 'chicken rice', 'Opinion': 'lacking.', 'Polarity': 'Negative'}, {'Aspect': 'service', 'Opinion': 'good', 'Polarity': 'Positive'}, {'Aspect': 'service', 'Opinion': 'lacking.', 'Polarity': 'Negative'}], 'True Triplets': []}


In [ ]:
config.MV.box_plot(no_overlap=False)
config.MV.scatter_plot()